<a href="https://www.kaggle.com/code/dlu913/cyclistic-python?scriptVersionId=144637351" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np
import seaborn as sns
import datetime 
import cufflinks as cf
from pandas.api.types import CategoricalDtype

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<h1>Ask</h1>

# Ask

The Business:

Cyclistic is a fictional public bike sharing company allowing both member and non-members to ride bikes on a rental-based model.  The bike-share program has more than 5,800 bikes and 600 docking stations.The data comes from the company itself from users using the service. The data  falls within the ROCCC framework.  It is licensed by Motivate International LLC.

Lily Moreno is a director of marketing for Cyclistic.  As part of the capstone project,  we will be gathering, cleaning and analyzing data in order to provide insights for business decisions.



<h2>Data</h2>

# Data
Import data from Kaggle. Data of each month is concated into one dataframe.

In [ ]:
#Import CSV files

Jan_2021 = pd.read_csv('../input/bikeset/202101-divvy-tripdata.csv')
Feb_2021 = pd.read_csv('../input/bikeset/202102-divvy-tripdata.csv')
March_2021 = pd.read_csv('../input/bikeset/202103-divvy-tripdata.csv')
April_2021 = pd.read_csv('../input/bikeset/202104-divvy-tripdata.csv')
May_2021 = pd.read_csv('../input/bikeset/202105-divvy-tripdata.csv')
June_2021 = pd.read_csv('../input/bikeset/202106-divvy-tripdata.csv')
July_2021 = pd.read_csv('../input/bikeset/202107-divvy-tripdata.csv')
Aug_2021 = pd.read_csv('../input/bikeset/202108-divvy-tripdata.csv')
Sep_2021 = pd.read_csv('../input/bikeset/202109-divvy-tripdata.csv')
Oct_2021 = pd.read_csv('../input/bikeset/202110-divvy-tripdata.csv')
Nov_2021 = pd.read_csv('../input/bikeset/202111-divvy-tripdata.csv')
Dec_2021 = pd.read_csv('../input/bikeset/202112-divvy-tripdata.csv')

In [ ]:
#Combine all CSV files into one dataset

bike_data = pd.concat([Jan_2021, Feb_2021, March_2021, April_2021, May_2021, June_2021, July_2021, Aug_2021, Sep_2021, Oct_2021, Nov_2021, Dec_2021], ignore_index = True)


bike_data






In [ ]:
bike_data.to_csv('mycsvfile.csv',index=False)

<h1>Data Cleaning</h1>

# Data Cleaning


In [ ]:
bike_data.info()

In [ ]:
 
bike_data.isnull().sum()

In [ ]:
#drop Null values
bike_data = bike_data.dropna(how='any',axis=0)

In [ ]:
bike_data.isnull().sum()

In [ ]:
#check for duplicates

bike_data.duplicated().any()

In [ ]:
# changing the 'started_at' and 'ended_at' dtype to datetime
bike_data['started_at']= bike_data['started_at'].astype('datetime64')
bike_data['ended_at']= bike_data['ended_at'].astype('datetime64')

In [ ]:
#Drop unnecessary columns

bike_data = bike_data.drop(columns=['start_station_name', 'start_station_id', 'end_station_name', 'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng'],axis='columns')

In [ ]:

# Creating New Column "ride_length" and changing its datatype to "int32"
# In this column, each row contains the difference between "starting time" and "ending time" columns in minutes.

bike_data['ride_length'] = (bike_data['ended_at'] - bike_data['started_at'])/pd.Timedelta(minutes=1)
bike_data['ride_length'] = bike_data['ride_length'].astype('int32')


bike_data.head()

In [ ]:
# Sorting Values by "ride_length" column in Ascending order.

bike_data.sort_values(by = 'ride_length')



In [ ]:
# Some months had too many negative values
# Such errors happened because the "ending time" is earlier than the "starting time" in their respective rows.

bike_data[bike_data['ride_length'] < 0].count()

# Number of rows containing Negative Values.

In [ ]:
bike_data[bike_data['ride_length'] < 1].count()

# Number of rows containing "ride length" less than "1" minute.


In [ ]:
bike_data = bike_data[bike_data['ride_length'] >= 1]
bike_data = bike_data.reset_index()
bike_data = bike_data.drop(columns=['index'])

# Removing 80845 rows containing negative values & ride length less than 1 minute. 
# Any trips that were below 60 seconds in length are potentially false starts or users trying to re-dock a bike to ensure it was secure.


In [ ]:
bike_data = bike_data.astype({'ride_id':'string', 'rideable_type':'category', 'member_casual':'category'})

# Coverting datatypes of each columns.

bike_data.info()

In [ ]:
# Checking if Dataframe "df" contains any missing values.

bike_data.isna().sum()

<h1>Analyze</h1>

In [ ]:
# Creating new columns "year", "month", "day_of_week", "hour" and Converting datatypes.


bike_data['year'] = bike_data['started_at'].dt.year

cats1 = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
bike_data['month'] = bike_data['started_at'].dt.month_name()
bike_data['month'] = bike_data['month'].astype(CategoricalDtype(categories=cats1, ordered=False))

cats2 = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
bike_data['day_of_week'] = bike_data['started_at'].dt.day_name()
bike_data['day_of_week'] = bike_data['day_of_week'].astype(CategoricalDtype(categories=cats2, ordered=False))

bike_data['hour'] = bike_data['started_at'].dt.hour

bike_data = bike_data.astype({'year':'int16', 'hour':'int8'})


bike_data.head()

In [ ]:
example_1 = bike_data.groupby('member_casual', as_index=False).count()


import plotly.express as px

px.bar(example_1, y = 'member_casual', x = 'ride_id', range_x = [0,3000000],
        color = 'member_casual', 
        height = 300,
        text = 'ride_id', 
        labels = {'ride_id': 'No. of Rides', 'member_casual': 'Member/Casual'},
        hover_name = 'member_casual', hover_data = {'member_casual': False, 'month': False, 'ride_id': True}, 
        color_discrete_map = {'casual': '#FF934F', 'member': '#058ED9'})

In [ ]:
#create pivot table 

bike_data_pv1 = pd.pivot_table(bike_data,
              index = ['year', 'month', 'member_casual'],
              values = 'ride_id',
              aggfunc = ['count'],
              margins = True,
              margins_name = 'Total Count')
bike_data_pv1 = bike_data_pv1.loc[(bike_data_pv1 != 0).any(axis=1)]
bike_data_pv1

In [ ]:
example_2 = bike_data.groupby(['year', 'month', 'member_casual'], as_index=False).count()

example_2 = example_2[example_2['ride_id'] != 0]

px.line(example_2, title = 'Riders Per Month', x = 'month', y = 'ride_id', range_y = [0,450000],
        color = 'member_casual', 
        line_shape = 'spline',
        markers=True, 
        labels = {'ride_id': 'No. of Rides', 'month': 'Months (Jan 2021 - Dec 2021)', 'member_casual': 'Member/Casual'},
        hover_name = 'member_casual', hover_data = {'member_casual': False, 'month': True, 'ride_id': True}, 
        color_discrete_map = {'casual': '#FF934F', 'member': '#058ED9'})

<h1>Average Number of Rides During Weekday</h1>

In [ ]:
pd.pivot_table(bike_data,
              index = ['day_of_week', 'member_casual'],
              values = 'ride_id',
              aggfunc = ['count'],
              margins = True,
              margins_name = 'Total Count')

In [ ]:
example_3 = bike_data.groupby(['day_of_week', 'member_casual'], as_index=False).count()

px.line(example_3, title= 'Average number of Riders per Weekday', x = 'day_of_week', y = 'ride_id', range_y = [0,600000],
        color = 'member_casual',
        line_shape = 'spline',
        markers=True,
        labels = {'ride_id': 'No. of Rides', 'day_of_week': 'Weekdays', 'member_casual': 'Member/Casual'},
        hover_name = 'member_casual', hover_data = {'member_casual': False, 'month': False, 'ride_id': True}, 
        color_discrete_map = {'casual': '#FF934F', 'member': '#058ED9'})



In [ ]:
# Bar graph of Average number of riders per weekday

example_3a = bike_data.groupby(['day_of_week', 'member_casual'], as_index=False).count()


example_3a = px.histogram(example_3a, title= 'Average number of Riders per Weekday', x = 'day_of_week', y = 'ride_id', range_y = [0,600000],
        color = 'member_casual',
        barmode='group',
        labels = {'ride_id': 'No. of Rides', 'day_of_week': 'Weekdays', 'member_casual': 'Member/Casual'},
        color_discrete_map = {'casual': '#FF934F', 'member': '#058ED9'})

example_3a.show()





<h1>Average Number of Rides per hour</h1>

In [ ]:
pd.pivot_table(bike_data,
              values = 'ride_id',
              index = ['hour', 'member_casual'],
              aggfunc = ['count'],
              margins = True,
              margins_name = 'Total Count')

In [ ]:
example_4 = bike_data.groupby(['hour', 'member_casual'], as_index=False).count()

example_4 = px.line(example_4, title = 'Number of Riders per Hour', x = 'hour', y = 'ride_id', range_x = [0,23], range_y = [0,350000],
                 color = 'member_casual',
                 line_shape = 'spline',
                 markers=True,
                 labels = {'ride_id': 'No. of Rides', 'hour': '24 Hours', 'member_casual': 'Member/Casual'},
                 hover_name = 'member_casual', hover_data = {'member_casual': False, 'month': False, 'ride_id': True},
                 color_discrete_map = {'casual': '#FF934F', 'member': '#058ED9'})

example_4.update_xaxes(dtick=1)



In [ ]:
#Average Ride Length Per Year

pd.pivot_table(bike_data,
               index = 'member_casual',
               values = ['ride_length'],
               aggfunc = ['mean'],
               margins = True,
               margins_name = 'Total Ride Average')



In [ ]:
#Calculate Mode of ride length between members and casual riders.
bike_data.groupby(['member_casual'])['ride_length'].agg(pd.Series.mode)

<h1>Average Ride Length</h1>

In [ ]:
example_5 = round(bike_data.groupby('member_casual', as_index=False).mean(),2)

px.bar(example_5, y = 'member_casual', x = 'ride_length', range_x = [0,35],
        color = 'member_casual', 
        height = 300,
        text = 'ride_length', 
        labels = {'ride_length': 'Average Ride Length (minutes)', 'member_casual': 'Member/Casual'},
        hover_name = 'member_casual', hover_data = {'member_casual': False, 'ride_length': True}, 
        color_discrete_map = {'casual': '#FF934F', 'member': '#058ED9'})

In [ ]:
#Average Ride Length Per month

pd.pivot_table(bike_data,
               index = ['year', 'month', 'member_casual'],
               values = ['ride_length'],
               aggfunc = ['mean'],
               margins = True,
               margins_name = 'Total Ride Average')

In [ ]:
example_6 = round(bike_data.groupby(['year', 'month', 'member_casual'], as_index=False).mean(),2).dropna()

px.bar(example_6,title = 'Average Ride Length per Month', x = 'month', y = 'ride_length',
        color = 'member_casual',
        barmode='group',
        text = 'ride_length', 
        labels = {'ride_length': 'Average Ride Length (minutes)', 'member_casual': 'Member/Casual', 'month': 'Months (Oct 2020 - Sep 2021)'},
        hover_name = 'member_casual', hover_data = {'member_casual': False, 'ride_length': True}, 
        color_discrete_map = {'casual': '#FF934F', 'member': '#058ED9'})

In [ ]:
#Average Ride Length per Weekday

pd.pivot_table(bike_data,
               index = ['day_of_week', 'member_casual'],
               values = ['ride_length'],
               aggfunc = ['mean'],
               margins = True,
               margins_name = 'Total Ride Average')

In [ ]:
example_7 = round(bike_data.groupby(['day_of_week', 'member_casual'], as_index=False).mean(),2)

px.bar(example_7, x = 'day_of_week', y = 'ride_length',
        color = 'member_casual',
        barmode='group',
        text = 'ride_length', 
        labels = {'ride_length': 'Average Ride Length (minutes)', 'member_casual': 'Member/Casual', 'day_of_week': 'Weekdays'},
        hover_name = 'member_casual', hover_data = {'member_casual': False, 'ride_length': True}, 
        color_discrete_map = {'casual': '#FF934F', 'member': '#058ED9'})

<h1>Conclusion</h1>

Members have a steady rate of riders during the weekday when people have to get to work.  Casual Riders spike on the weekend. There is a spike of both casual and member riders during the morning (around 9pm) and evening hours (around 6pm) when people are commuting to and off work.  However,  there are still more member riders during these times.

Recommendations

As a business strategy,  doubling down on casual riders on the weekend would create a great source of continued revenue.  Increasing marketing campaigns and incentives to get more people to ride on weeekends would be a viable move for  attracting more riders. 

Scaling more riders in with multiple ridership packages can be an attractive model.  A second order effect of converting them to members would be a good pivot (incentivized cheaper rates for members).

A gamification system of giving riders perks based on rider usage could be a new alluring model.  Rewards such as free miles can be an attractive perk.  Members can also spare or give miles to other family members or friends to use.

It would be of interest to view transportation trends such as other modes of public transportation and traffic routes.  Is there a way to incentivize people to ride bikes during peak traffic hour?

In [ ]:
bike_data.to_csv('mycsvfile.csv',index=False)